In [1]:
import json
import pandas as pd

In [2]:
k = 3

data = pd.read_csv("generation\\database.csv")

numerical_qis = ['age']
categorical_qis = ['gender', 'city', 'education', 'profession']
all_qis = categorical_qis + numerical_qis
explicit_identifier = ['person_id', 'first_name', 'last_name']
sensitive_data = ['annual_income']
statistic = "range"

In [3]:
# Function to load JSON data from file
def load_json(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

In [4]:
# Function to drop specified columns from a DataFrame
def drop_EI(df, EI):
    return df.drop(columns=EI)

In [5]:
import json

def find_parents(node, target, path=None, key=None):
    """Trova tutti i genitori di un nodo target in un albero JSON.

    Args:
        node (dict or list): Nodo dell'albero JSON.
        target (str): Nodo target da cercare.
        path (list, optional): Percorso attuale nell'albero JSON. Default: None.
        key (str, optional): Chiave per cercare il nodo target all'interno di un dizionario. Default: None.

    Returns:
        list or None: Lista di nomi dei genitori del nodo target oppure None se non trovato.
    """
    if path is None:
        path = []

    current_name = node.get('name', None) if isinstance(node, dict) else None

    if isinstance(node, dict):
        # Caso in cui il nodo è un dizionario
        if key and node.get(key) == target:
            # Se è stato trovato il nodo target all'interno del dizionario
            return path + [current_name]

        for k, v in node.items():
            if isinstance(v, list) and target in v:
                # Se il nodo target è presente nella lista
                return path + [current_name]
            elif isinstance(v, (dict, list)):
                # Ricorsivamente cerca nei sotto-nodi
                result = find_parents(v, target, path + [current_name] if current_name else path, key)
                if result:
                    return result
    elif isinstance(node, list):
        # Caso in cui il nodo è una lista
        for item in node:
            if isinstance(item, (dict, list)):
                # Ricorsivamente cerca nei sotto-nodi
                result = find_parents(item, target, path, key)
                if result:
                    return result
            elif item == target:
                # Se il nodo target è stato trovato nella lista
                return path + [current_name]

    return None

def find_target_parents_in_json(file_path, target, key=None):
    """Trova tutti i genitori di un nodo target in un file JSON specificato.

    Args:
        file_path (str): Percorso del file JSON.
        target (str): Nodo target da cercare.
        key (str, optional): Chiave per cercare il nodo target all'interno di un dizionario. Default: None.

    Returns:
        list or None: Lista di nomi dei genitori del nodo target oppure None se non trovato.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)

    return find_parents(data, target, key=key)


In [6]:
def find_lowest_common_ancestor(file_path, target1, target2, key=None):
    """Trova l'antenato comune più basso di due nodi target in un file JSON.

    Args:
        file_path (str): Percorso del file JSON.
        target1 (str): Primo nodo target.
        target2 (str): Secondo nodo target.
        key (str, optional): Chiave per cercare il nodo target all'interno di un dizionario. Default: None.

    Returns:
        str or None: Nome dell'antenato comune più basso oppure None se non trovato.
    """
    parents1 = find_target_parents_in_json(file_path, target1, key)
    parents2 = find_target_parents_in_json(file_path, target2, key)

    if not parents1 or not parents2:
        return None

    # Rimuove i valori None e deduplica le liste di genitori
    parents1 = [parent for parent in parents1 if parent]
    parents2 = [parent for parent in parents2 if parent]

    # Trova l'antenato comune più basso confrontando i percorsi
    lca = None
    for p1, p2 in zip(parents1, parents2):
        if p1 == p2:
            lca = p1
        else:
            break

    return lca

# Esempi di utilizzo

# jobs.json
file_path_jobs = 'generation/jobs.json'
target1_jobs = 'Bridge Engineer'
target2_jobs = 'Highway Engineer'
lca_jobs = find_lowest_common_ancestor(file_path_jobs, target1_jobs, target2_jobs)
print(f"L'antenato comune più basso di '{target1_jobs}' e '{target2_jobs}' è: {lca_jobs}")

# educations.json
file_path_educations = 'generation/educations.json'
target1_educations = "Master's Degree"
target2_educations = "Doctoral Degree"
lca_educations = find_lowest_common_ancestor(file_path_educations, target1_educations, target2_educations, key='name')
print(f"L'antenato comune più basso di '{target1_educations}' e '{target2_educations}' è: {lca_educations}")

# cities.json
file_path_cities = 'generation/cities.json'
target1_cities = 'Boston'
target2_cities = 'Chicago'
lca_cities = find_lowest_common_ancestor(file_path_cities, target1_cities, target2_cities)
print(f"L'antenato comune più basso di '{target1_cities}' e '{target2_cities}' è: {lca_cities}")

# gender.json
file_path_gender = 'generation/genders.json'
target1_gender = 'Male'
target2_gender = 'Female'
lca_gender = find_lowest_common_ancestor(file_path_gender, target1_gender, target2_gender, key='name')
print(f"L'antenato comune più basso di '{target1_gender}' e '{target2_gender}' è: {lca_gender}")

L'antenato comune più basso di 'Bridge Engineer' e 'Highway Engineer' è: Structural Engineer
L'antenato comune più basso di 'Master's Degree' e 'Doctoral Degree' è: Graduate School
L'antenato comune più basso di 'Boston' e 'Chicago' è: USA
L'antenato comune più basso di 'Male' e 'Female' è: ANY-GENDER


In [7]:
def splitter(dataframe, column, k):
    """Splits the dataframe along a certain column respecting k value"""
    if column in dataframe.select_dtypes(include='number').columns:
        median_value = dataframe[column].median()
        left_partition = dataframe[dataframe[column] <= median_value]
        right_partition = dataframe[dataframe[column] > median_value]
    elif column in dataframe.select_dtypes(include='object').columns:
        unique_values = dataframe[column].unique()
        middle_index = len(unique_values) // 2
        middle_value = unique_values[middle_index]
        left_partition = dataframe[dataframe[column] <= middle_value]
        right_partition = dataframe[dataframe[column] > middle_value]
    else:
        # Skip columns that are neither numeric nor categorical
        return None, None
    
    if len(left_partition) >= k and len(right_partition) >= k:
        return left_partition, right_partition
    else:
        # Adjust partitions if lengths are less than k
        left_partition = dataframe.iloc[:k]
        right_partition = dataframe.iloc[k:]
        return left_partition, right_partition


In [8]:
# Esegui lo split per ogni colonna in all_qis
for column in all_qis:
    left, right = splitter(data, column, k)
    if left is not None and right is not None:
        print(f"Left partition shape for column '{column}': {left.shape}")
        print(f"Right partition shape for column '{column}': {right.shape}")

Left partition shape for column 'gender': (490, 9)
Right partition shape for column 'gender': (510, 9)
Left partition shape for column 'city': (679, 9)
Right partition shape for column 'city': (321, 9)
Left partition shape for column 'education': (494, 9)
Right partition shape for column 'education': (506, 9)
Left partition shape for column 'profession': (365, 9)
Right partition shape for column 'profession': (635, 9)
Left partition shape for column 'age': (504, 9)
Right partition shape for column 'age': (496, 9)


In [9]:
def recursive_partition(dataset, k, sensitive_data):
    """Splits the dataset in partitions recursively."""
    
    def axe_to_split(dataframe, sensitive_data):
        # Find column with highest cardinality (unique values) to split on
        columns_to_exclude = [col for col in dataframe.columns if col in sensitive_data]
        max_cardinality_column = dataframe.drop(columns_to_exclude, axis=1).nunique().idxmax()
        return max_cardinality_column
    
    # Base case: if dataset size is smaller than k*2, add it to partitions list
    if len(dataset) < k * 2:
        dataframe_partitions.append(dataset)
    else:
        # Split according to column with highest cardinality
        axe = axe_to_split(dataset, sensitive_data)
        left_partition, right_partition = splitter(dataset, axe, k)
        
        # Recursively partition left and right partitions
        recursive_partition(left_partition, k, sensitive_data)
        recursive_partition(right_partition, k, sensitive_data)

# Example usage:
dataframe_partitions = []
k = 3
sensitive_data = ['annual_income']  # List of sensitive columns to exclude from splitting
data = drop_EI(data, explicit_identifier)
recursive_partition(data, k, sensitive_data)

# Stampa il numero di partizioni create
print(f"Numero di partizioni: {len(dataframe_partitions)}")



Numero di partizioni: 289


In [10]:
def generalize_partition(partition, qis, json_files, statistic):
    """Generalizza una partizione del dataset sostituendo i valori dei quasi-identificatori con la loro generalizzazione.
        Se il quasi-identificatore è numerico, viene generalizzato con il range di valori o la media.
        Se il quasi-identificatore è categorico, viene generalizzato con l'antenato comune più basso (LCA).

    Args:
        partition (DataFrame): Partizione del dataset da generalizzare.
        qis (list): Lista dei quasi-identificatori da considerare.
        json_files (dict): Dizionario contenente i percorsi ai file JSON per i QI categorici.
        statistic (str): Metodo di generalizzazione per i quasi-identificatori numerici ('range' o 'mean').

    Returns:
        DataFrame: Partizione del dataset generalizzata.
    """
    for qi in qis:
        # Ordina la partizione in base al quasi-identificatore corrente
        partition = partition.sort_values(by=qi)
        
        # Controlla se tutti i valori sono uguali per il quasi-identificatore corrente
        if partition[qi].iloc[0] != partition[qi].iloc[-1]:
            if qi in numerical_qis:
                if statistic == 'range':
                    # Se il quasi-identificatore è numerico, generalizza con il range di valori
                    min_val = partition[qi].iloc[0]
                    max_val = partition[qi].iloc[-1]
                    s = f"[{min_val} - {max_val}]"
                elif statistic == 'mean':
                    # Se il quasi-identificatore è numerico e la statistica è 'mean', generalizza con la media
                    mean_val = partition[qi].mean()
                    s = f"[{mean_val}]"
                else:
                    raise ValueError("Statistic must be 'range' or 'mean'")
            else:
                # Se il quasi-identificatore è categorico, cerca l'antenato comune più basso (LCA)
                unique_values = sorted(set(partition[qi]))
                if len(unique_values) == 1:
                    lca = unique_values[0]
                else:
                    lca = unique_values[0]
                    for value in unique_values[1:]:
                        lca_candidate = find_lowest_common_ancestor(json_files[qi], lca, value, key='name')
                        if lca_candidate:
                            lca = lca_candidate
                        else:
                            lca = 'ANY'
                            break
                        
                s = f"[{lca}]"
            
            # Sostituisce i valori del quasi-identificatore con la generalizzazione trovata
            partition[qi] = [s] * partition[qi].size
    
    return partition


In [11]:
json_files = {
    'city': 'generation/cities.json',
    'profession': 'generation/jobs.json',
    'education': 'generation/educations.json',
    'gender': 'generation/genders.json'
}

In [13]:
def mondrian(database, k, qis, sd, json_files):
    anonymized_data = recursive_partition(database, k, sd)
    generalized_partitions = []
    for i, partition in enumerate(dataframe_partitions):
        generalized_partition = generalize_partition(partition, qis, json_files, statistic)
        generalized_partitions.append(generalized_partition)
    anonymized_data = pd.concat(generalized_partitions)
    anonymized_data.to_csv('anonymized.csv', index=False)
    print("Dati anonimizzati salvati in anonymized.csv")

In [14]:
anonymized_data = mondrian(data, k, all_qis, sensitive_data, json_files)

Dati anonimizzati salvati in anonymized.csv
